# Importing the Data

In [19]:
import re

# to handle datasets
import pandas as pd
import numpy as np

# for visualization
import matplotlib.pyplot as plt

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import StandardScaler

# to build the models
from sklearn.linear_model import LogisticRegression

# to evaluate the models
from sklearn.metrics import accuracy_score, roc_auc_score

# to persist the model and the scaler
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)
pd.pandas.set_option('display.max_rows', 100)

In [3]:
# load the csvs - it is available open source and online
iaps = pd.read_csv('iaps.csv')
sessions = pd.read_csv('sessions.csv')
spendevents = pd.read_csv('spendevents.csv')
users = pd.read_csv('users.csv')

In [4]:
print(len(iaps))
print(len(sessions))
print(len(spendevents))
print(len(users))

6685
722955
107764
22576


# Exploratory Analysis

## IAPS Analysis

In [15]:
# display data
iaps

,user_id,ts,date,prod_name,prod_type,rev
0,7480,2019-03-04 08:15:49,2019-03-04,iap_1_gems_2,gems,760
1,7480,2019-03-04 08:24:15,2019-03-04,iap_1_passes_2,chapterPasses,760
2,7480,2019-03-04 22:49:08,2019-03-04,iap_1_gems_1,gems,410
3,2466,2019-03-06 00:16:48,2019-03-06,iap_1_gems_2,gems,760
4,22001,2019-03-06 09:13:45,2019-03-06,iap_1_gems_2,gems,760
...,...,...,...,...,...,...
6680,18174,2019-04-21 18:50:42,2019-04-21,iap_1_passes_10,chapterPasses,3555
6681,18174,2019-04-21 18:51:29,2019-04-21,iap_1_passes_1,chapterPasses,410
6682,18174,2019-04-21 20:08:06,2019-04-21,iap_1_passes_10,chapterPasses,3555
6683,18174,2019-04-22 17:07:19,2019-04-22,iap_1_passes_2,chapterPasses,760


In [14]:
iaps.dtypes

user_id       int64
ts           object
date         object
prod_name    object
prod_type    object
rev           int64
dtype: object

#### Analysis of how much each product costs

In [26]:
# These are all the packages the users can buy
iaps['prod_name'].value_counts()

iap_1_gems_2          2599
iap_1_passes_2        1064
iap_1_gems_5           976
iap_1_gems_1           693
iap_1_passes_1         576
iap_1_gems_10          231
iap_1_passes_5         226
iap_1_passes_10        163
iap_1_gems_20           69
iap_value_pack_004      46
iap_value_pack_008      13
iap_value_pack_001      12
iap_value_pack_005       5
iap_1_gems_50            3
iap_value_pack_010       3
iap_1_gems_100           2
iap_value_pack_003       2
iap_value_pack_009       1
iap_value_pack_007       1
Name: prod_name, dtype: int64

In [7]:
# These are the amount of gems that the user bought. These should correspond to the prod_name packages
iaps['rev'].value_counts()

760      3663
410      1281
1810     1207
3560      234
3555      163
7060       69
1460       46
2860       13
17560       3
35060       2
1110        2
2510        1
3210        1
Name: rev, dtype: int64

In [10]:
iaps['rev'][2]

410

In [13]:
prod_list = {}

for i, val in enumerate(iaps['prod_name']):
    if val not in prod_list:
        prod_list[val] = iaps['rev'][i]
        # print(val, iaps['rev'][i])

prod_list

{'iap_1_gems_2': 760,
 'iap_1_passes_2': 760,
 'iap_1_gems_1': 410,
 'iap_1_passes_1': 410,
 'iap_1_gems_5': 1810,
 'iap_1_passes_5': 1810,
 'iap_1_gems_10': 3560,
 'iap_1_passes_10': 3555,
 'iap_1_gems_20': 7060,
 'iap_value_pack_004': 1460,
 'iap_value_pack_008': 2860,
 'iap_1_gems_50': 17560,
 'iap_value_pack_001': 410,
 'iap_value_pack_005': 1810,
 'iap_value_pack_003': 1110,
 'iap_value_pack_010': 3560,
 'iap_1_gems_100': 35060,
 'iap_value_pack_007': 2510,
 'iap_value_pack_009': 3210}

#### Date Analysis

In [16]:
# Purchase data seems to be from March to May in 2019
iaps.sort_values(by=['ts'])

,user_id,ts,date,prod_name,prod_type,rev
3460,19248,2019-03-01 00:24:52,2019-03-01,iap_1_gems_2,gems,760
3461,19248,2019-03-01 00:36:14,2019-03-01,iap_1_gems_2,gems,760
3442,12752,2019-03-01 01:18:04,2019-03-01,iap_1_passes_1,chapterPasses,410
2074,10387,2019-03-01 01:19:41,2019-03-01,iap_1_gems_2,gems,760
5431,13356,2019-03-01 03:35:04,2019-03-01,iap_1_passes_2,chapterPasses,760
...,...,...,...,...,...,...
2963,20401,2019-05-05 17:26:24,2019-05-05,iap_1_passes_2,chapterPasses,760
3604,15446,2019-05-05 18:04:08,2019-05-05,iap_value_pack_010,valuePack,3560
5430,15443,2019-05-05 20:40:23,2019-05-05,iap_1_passes_10,chapterPasses,3555
3605,15446,2019-05-05 22:12:51,2019-05-05,iap_1_gems_2,gems,760


In [20]:
# There seems to be less data as time goes on
iaps['date'].value_counts()

2019-03-06    873
2019-03-07    766
2019-03-05    653
2019-03-08    451
2019-03-04    433
2019-03-03    356
2019-03-02    333
2019-03-09    260
2019-03-01    188
2019-03-10    173
2019-03-11    155
2019-03-12    131
2019-03-13    131
2019-03-19    113
2019-03-20     91
2019-03-14     83
2019-03-18     64
2019-03-27     63
2019-03-16     59
2019-04-02     56
2019-03-15     52
2019-04-03     50
2019-03-29     49
2019-03-26     49
2019-03-25     48
2019-03-21     44
2019-04-09     44
2019-03-28     43
2019-03-17     43
2019-03-31     42
2019-03-24     42
2019-04-16     42
2019-03-22     39
2019-04-04     38
2019-04-01     37
2019-04-22     34
2019-04-17     32
2019-04-06     30
2019-04-21     30
2019-04-14     30
2019-03-23     30
2019-04-05     29
2019-04-13     27
2019-04-12     26
2019-04-15     26
2019-03-30     23
2019-04-08     23
2019-04-10     21
2019-04-11     21
2019-04-07     21
2019-04-30     20
2019-04-23     19
2019-04-27     19
2019-04-24     17
2019-04-20     17
2019-04-28

## Spendevents Exploratory Analysis

In [32]:
# display data
spendevents

,user_id,ts,date,story,chapter,spendtype,currency,amount
0,9829,2019-03-01 03:03:04,2019-03-01,story_1,0,earnGemsCounter,gems,-22
1,13757,2019-03-01 03:35:53,2019-03-01,story_1,0,earnGemsCounter,gems,-22
2,13757,2019-03-01 03:52:10,2019-03-01,story_2,0,earnGemsCounter,gems,-22
3,10009,2019-03-01 04:10:00,2019-03-01,story_1,0,earnGemsCounter,gems,-22
4,10009,2019-03-01 04:26:46,2019-03-01,story_2,0,earnGemsCounter,gems,-22
...,...,...,...,...,...,...,...,...
107759,15156,2019-04-23 02:31:46,2019-04-23,story_3,0,earnGemsCounter,gems,0
107760,15156,2019-04-23 02:31:59,2019-04-23,story_3,0,earnGemsCounter,gems,0
107761,15156,2019-04-23 02:32:04,2019-04-23,story_3,0,earnGemsCounter,gems,0
107762,15156,2019-04-23 02:32:11,2019-04-23,story_3,0,earnGemsCounter,gems,0


In [37]:
# filter to user id == 2062 due to assignment instructions
filtered_spendevents = spendevents['user_id'] == 2062

# order by ts so we can see all three spendtypes (according to assignment instructions)
spendevents[filtered_spendevents].sort_values(by=['ts'])

,user_id,ts,date,story,chapter,spendtype,currency,amount
15694,2062,2019-03-02 20:01:22,2019-03-03,story_1,0,earnGemsCounter,gems,-22
15721,2062,2019-03-03 03:50:45,2019-03-03,story_1,5,IAP,gems,-73
15722,2062,2019-03-03 03:50:58,2019-03-03,story_1,5,IAP,gems,-73
15723,2062,2019-03-03 03:51:00,2019-03-03,story_1,5,premiumChoice,gems,64
15724,2062,2019-03-03 04:07:34,2019-03-03,story_1,6,IAP,gems,-73
15725,2062,2019-03-03 04:07:37,2019-03-03,story_1,6,premiumChoice,gems,55
15726,2062,2019-03-03 04:24:58,2019-03-03,story_1,8,premiumChoice,gems,27


In [39]:
# the only currency is gems. This column is irrelevant
spendevents['currency'].value_counts()

gems    107764
Name: currency, dtype: int64

In [23]:
# shows that there is a 4th option for spendtype that wasn't discussed in the assignment (valuepack)
spendevents['spendtype'].value_counts()

earnGemsCounter    87235
premiumChoice      14231
IAP                 4915
valuepack             39
Name: spendtype, dtype: int64

In [41]:
# filter to spendtype == valuepack
filtered_spendtype = spendevents['spendtype'] == 'valuepack'

# we can see that value pack is another pack where the user spends money to buy gems
spendevents[filtered_spendtype].head()

,user_id,ts,date,story,chapter,spendtype,currency,amount
27107,9984,2019-04-15 20:16:49,2019-04-16,story_15,10,valuepack,gems,-322
32772,5782,2019-04-17 05:33:26,2019-04-17,story_1,4,valuepack,gems,-82
52405,5987,2019-04-17 05:02:48,2019-04-17,story_4,24,valuepack,gems,-217
52406,5987,2019-04-17 05:02:48,2019-04-17,story_4,24,valuepack,gems,-217
57519,8874,2019-04-17 07:46:03,2019-04-17,story_2,0,valuepack,gems,-217


# Sessions Exploratory Analysis

In [7]:
# display data
sessions.head()

,user_id,ts,date,session_num,last_session_termination_type
0,14067,2019-03-01 00:06:50,2019-03-01,1,NaN
1,14067,2019-03-01 00:22:27,2019-03-01,2,NaN
2,16275,2019-03-01 01:23:03,2019-03-01,1,NaN
3,16275,2019-03-01 01:31:16,2019-03-01,2,NaN
4,16275,2019-03-01 01:47:22,2019-03-01,3,NaN


In [9]:
# display data
users.head()

,user_id,install_date,lang,country,hw_ver,os_ver
0,0,2019-03-01,en,US,"iPhone4,1",9.1
1,1,2019-03-01,en,IN,"iPod5,1",8.1.2
2,2,2019-03-06,en,US,"iPod7,1",8.4.1
3,3,2019-03-03,nb,NO,"iPhone8,1",9.2.1
4,4,2019-03-03,en,GB,"iPhone5,4",9.2.1


In [16]:
df1 = pd.merge(iaps, users, on='user_id', how='outer')
print(len(df1))

27735


In [ ]:
df = pd.merge(iaps, sessions, spendevents, users, on='user_id', how='outer')